In [1]:
# Import necessary libraries
import asyncio
import sys
import os
import warnings
from devtools import pprint
# Add the src directory to the Python path and suppress warnings
sys.path.append(os.path.abspath('../../../src/agent'))
warnings.filterwarnings("ignore", category=FutureWarning)
from market_intelligence_agent.config import TEAM_MEMBERS
from langchain_core.messages import convert_to_messages
from langgraph_sdk import get_client
from datetime import datetime

import logging
import uuid


In [2]:

# LangGraph Client
lg_client = get_client(url="http://localhost:2024")

user_query = ["What are Nvidia's performance over the past month? You should plot the price and volume over the past month. Justify the performance based on the news"]
user_input_messages = user_query

thread = await lg_client.threads.create()

print("Starting graph stream...")
# NOTE: The input dictionary is simplified
async for chunk in lg_client.runs.stream(
    thread["thread_id"],
    assistant_id="market_intelligence_agent",
    input={
        # Constants
        "TEAM_MEMBERS": TEAM_MEMBERS,
        # Runtime Variables
        "messages": user_input_messages,
        # "mcp_tools": mcp_tools_global, # REMOVED
        "deep_thinking_mode": True,
        "search_before_planning": True,
        "current_timestamp": datetime.now()
    },
    stream_mode="values"
):
    messages = chunk.data.get("messages", None)
    if messages:
        pprint(convert_to_messages(messages)[-1])
    print("=" * 50)

print("Graph stream finished.")



Starting graph stream...
HumanMessage(
    content=(
        "What are Nvidia's performance over the past month? You should plot the price and volume over the past month. "
        'Justify the performance based on the news'
    ),
    additional_kwargs={},
    response_metadata={},
    id='37658bdf-7532-40f1-a340-5713a5e4b9df',
)
HumanMessage(
    content=(
        '{\n'
        '  "thought": "The user wants to understand how Nvidia (NVDA) stock has performed over the last month, illustr'
        'ated with a price‑and‑volume plot, and an explanation of that performance grounded in recent news.",\n'
        '  "title": "Nvidia One‑Month Performance Analysis",\n'
        '  "steps": [\n'
        '    {\n'
        '      "agent_name": "researcher",\n'
        '      "title": "Collect and summarize news on Nvidia from the past month",\n'
        '      "description": "Search financial news sources, press releases, and analyst commentary from the last 30 '
        'days to identify major 

In [3]:
current_state = await lg_client.threads.get_state(thread["thread_id"])

In [4]:
current_state['values']['messages']

[{'content': "What are Nvidia's performance over the past month? You should plot the price and volume over the past month. Justify the performance based on the news",
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'human',
  'name': None,
  'id': '37658bdf-7532-40f1-a340-5713a5e4b9df',
  'example': False},
 {'content': '{\n  "thought": "The user wants to understand how Nvidia (NVDA) stock has performed over the last month, illustrated with a price‑and‑volume plot, and an explanation of that performance grounded in recent news.",\n  "title": "Nvidia One‑Month Performance Analysis",\n  "steps": [\n    {\n      "agent_name": "researcher",\n      "title": "Collect and summarize news on Nvidia from the past month",\n      "description": "Search financial news sources, press releases, and analyst commentary from the last 30 days to identify major events that could influence Nvidia’s stock price (e.g., earnings release, product announcements, AI‑chip market news, regulatory i

In [5]:
pprint(current_state['values']['messages'][-1])

{
    'content': (
        'Response from reporter:\n'
        '\n'
        '<response>\n'
        '# Nvidia (NVDA) One-Month Performance Report\n'
        '\n'
        '## Executive Summary\n'
        '\n'
        'Over the past month (March 19, 2025 – April 17, 2025), Nvidia’s stock (NVDA) experienced significant volatili'
        'ty, declining by approximately 13.5%. The period was marked by sharp price drops, high trading volumes, and a'
        ' partial recovery. The primary drivers of this performance were negative regulatory developments, competitive'
        ' pressures, and broader weakness in the technology sector, partially offset by positive product announcements'
        ' at Nvidia’s GTC 2025 conference.\n'
        '\n'
        '---\n'
        '\n'
        '## Key Findings\n'
        '\n'
        '- **Price Performance:** NVDA fell from $117.27 (first open) to $101.49 (last close), a decline of -13.46%.\n'
        '- **Volatility:** The stock experienced an annualized v

In [6]:
from IPython.display import Markdown, display

def display_markdown(content):
    start_marker = "<response>\n"
    end_marker = "</response>\n"
    start_index = content.find(start_marker)
    end_index = content.find(end_marker)
    if start_index != -1 and end_index != -1:
        content = content[start_index + len(start_marker):end_index]
    return Markdown(content)
    
    
# Display the formatted markdown
display(display_markdown(current_state['values']['messages'][-1]['content']))

# Nvidia (NVDA) One-Month Performance Report

## Executive Summary

Over the past month (March 19, 2025 – April 17, 2025), Nvidia’s stock (NVDA) experienced significant volatility, declining by approximately 13.5%. The period was marked by sharp price drops, high trading volumes, and a partial recovery. The primary drivers of this performance were negative regulatory developments, competitive pressures, and broader weakness in the technology sector, partially offset by positive product announcements at Nvidia’s GTC 2025 conference.

---

## Key Findings

- **Price Performance:** NVDA fell from $117.27 (first open) to $101.49 (last close), a decline of -13.46%.
- **Volatility:** The stock experienced an annualized volatility of 89.16% and a maximum drawdown of -22.32%.
- **Volume:** Average daily trading volume was 326.95 million shares, with total volume over the month at 7.19 billion shares. Volume spiked during periods of sharp price decline.
- **News Impact:** The most significant negative catalyst was the announcement of new US export restrictions on Nvidia’s AI chips to China, which threatened a substantial portion of the company’s earnings. Additional pressure came from competitive advances by Huawei and analyst downgrades. Positive sentiment was generated by major product launches at GTC 2025 and reassurances from Nvidia’s CEO.

---

## Detailed Analysis

### 1. Price & Volume Chart

![Nvidia Price and Volume Chart](output/plots/plot_5d18494f-8b0d-4dde-aad4-2906bdceff0e.png)

- The chart (see above) shows a sharp decline from late March highs (~$122) to early April lows (~$86), followed by a partial recovery to ~$101 by mid-April.
- Volume bars spike during the steepest price drops, indicating heavy trading activity and likely institutional repositioning.

### 2. Key Metrics

| Metric                    | Value                |
|---------------------------|----------------------|
| First Open Price          | $117.27              |
| Last Close Price          | $101.49              |
| Overall Change            | -13.46%              |
| Highest Price             | $122.22              |
| Lowest Price              | $86.62               |
| Average Daily Volume      | 326.95M shares       |
| Total Volume              | 7.19B shares         |
| Volatility (annualized)   | 89.16%               |
| Max Drawdown              | -22.32%              |

### 3. News Events and Their Impact

| Date         | Headline & Source                                                                 | Summary & Sentiment                                                                                  |
|--------------|-----------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------|
| Apr 18-19    | US restricts Nvidia AI chip exports to China (Techmeme, Benzinga, Yahoo Finance)  | New export bans on Nvidia’s H20 AI chip to China could cut earnings by $5.5B. Shares tumbled ~8.5%. **Negative** |
| Apr 18       | Huawei debuts AI system faster than Nvidia’s NVL72 (TechRadar)                     | Huawei’s new AI system outperforms Nvidia’s flagship in China, raising competitive pressure. **Negative** |
| Apr 18       | Nvidia GTC 2025: 18 new products announced (The New Stack)                         | Major product launches (Blackwell GPUs, DGX Spark, new AI/ML hardware) highlight innovation. **Positive** |
| Apr 18       | Nvidia CEO stresses commitment to China (GuruFocus)                                | CEO reassures investors about China market focus amid export bans. **Neutral/Positive**               |
| Apr 18       | Nvidia caught in “Category 5 tariff storm” (Benzinga)                             | Analysts warn of escalating US-China tech trade war risks. **Negative**                               |
| Apr 18       | Jim Cramer: NVDA is a “half-trillion chip bet” (WealthCreationInvesting)           | Cramer remains optimistic on Nvidia’s long-term prospects despite volatility. **Positive**            |
| Apr 18       | TSMC’s AI chip profits rise (NextPlatform)                                        | TSMC’s gains reflect ongoing AI demand, indirectly positive for Nvidia’s ecosystem. **Neutral**       |
| Apr 7-17     | Market-wide tech selloff, analyst downgrades (Yahoo Finance, Benzinga)             | Broader tech sector weakness and price target cuts contributed to NVDA’s drawdown. **Negative**       |

#### News-Price Linkage

- **Sharp Decline:** The steepest price drop and volume spike occurred after the US announced new export restrictions on Nvidia’s AI chips to China, threatening a $5.5B hit to earnings.
- **Partial Recovery:** The GTC 2025 conference and CEO reassurances provided some support, but ongoing geopolitical and competitive pressures limited the rebound.
- **Broader Market:** Sector-wide tech weakness and analyst downgrades amplified the negative sentiment.

---

## Conclusions and Recommendations

- **Conclusion:** Nvidia’s -13.5% monthly decline was primarily driven by regulatory headwinds (US-China export restrictions), competitive threats (notably from Huawei), and broader tech sector volatility. Positive product news and management reassurances provided only partial relief.
- **Outlook:** Near-term volatility is likely to persist as geopolitical and competitive risks remain elevated. However, Nvidia’s continued innovation and strong product pipeline may support long-term growth prospects.

---

**Sources:**  
- Price & volume data: Polygon API (2025-03-19 to 2025-04-17)  
- News: Techmeme, Benzinga, Yahoo Finance, The New Stack, TechRadar, GuruFocus, WealthCreationInvesting, NextPlatform

*If you require further breakdowns or additional visualizations, please specify your request.*
